### Strategy
Since Mostly the portfolio fails if the withdrawls happens during Market Crash,
We can use the Debt to make sure the withdrawls are not happening during crash period.

Logic:
Withdraw from equity if the value of equity is more than inflation adjusted appreciation
else Withdraw from debt.

In [ ]:
import pandas as pd 
import math
import plotly.express as px


In [ ]:
sensex_1980 = "../data/SnP-Sensex-1980.csv"
sensex_df = pd.read_csv(sensex_1980, header=0)
sensex_df
del sensex_df["Low"]
del sensex_df["Open"]
del sensex_df["High"]
sensex_df["Returns"] = sensex_df["Close"].pct_change()
sensex_df= sensex_df.fillna(0)

In [ ]:
sensex_df

#### Effective Return = Return On Equity + Return on Debt - Inflated Value fo Withdrawl

In [ ]:
DEBT_RETURN = 1.09
INFLATION_ANNUAL = 1.05
SWR = 4
SWR_RANGES = [0,1,2,3,4,5]
MONTHLY_DEBT_RETURN = math.pow(DEBT_RETURN, 1/12) - 1
MONTHLY_INFLATION = math.pow(INFLATION_ANNUAL, 1/12) - 1
FIRST_WITHDRAWAL = SWR/12
EQUITY_SHARE = .7
DEBT_SHARE = 1 - EQUITY_SHARE
INITIAL_NVESTMENT = 100
DATAPOINTS = len(sensex_df.index)
EXCLUDE_RESULT = 60 # Last 1 year , result will excluded for performance calculation

In [ ]:
def calculate_withdrawls(initial_withdrawal):
    withdrawals = list()
    withdrawals_values = initial_withdrawal
    for i in range(DATAPOINTS):
        withdrawals_values = (1+MONTHLY_INFLATION)*withdrawals_values
        withdrawals.append(withdrawals_values)
    return withdrawals


In [ ]:
def calculate_inflated_portfolio(initial_investment):
    inflated_portfolio = list()
    inflated_values = initial_investment
    inflated_portfolio.append(initial_investment)
    for i in range(DATAPOINTS):
        inflated_values = (1+MONTHLY_INFLATION)*inflated_values
        inflated_portfolio.append(inflated_values)
    return inflated_portfolio
INFLATED_PORTFOLIO = calculate_inflated_portfolio(INITIAL_NVESTMENT*EQUITY_SHARE)

In [ ]:
def generate_portfolio(main_index, withdrawals):
    equity = INITIAL_NVESTMENT*EQUITY_SHARE
    debt = INITIAL_NVESTMENT*DEBT_SHARE
    current_month_amount = equity + debt
    monthly_portfolio_value = list()
    for index, row in sensex_df.iterrows():
        if main_index < index:
            if equity >= INFLATED_PORTFOLIO[index]: ## Deduct if equitu is more than value of infaltion adjusted
                debt = (1+ MONTHLY_DEBT_RETURN)*debt
                equity = (1 + row["Returns"])*equity - withdrawals[index]
            else:
                debt = (1+ MONTHLY_DEBT_RETURN)*debt - withdrawals[index]
                equity = (1 + row["Returns"])*equity 
            total_value = equity + debt
            monthly_portfolio_value.append(total_value)
        elif main_index == index:
            monthly_portfolio_value.append(equity + debt)
        else:
            monthly_portfolio_value.append(None)
    return monthly_portfolio_value

In [ ]:
performance_at_swr = dict()
for swr in SWR_RANGES:
    initial_withdrawal = swr/12
    withdrawals = calculate_withdrawls(initial_withdrawal)
    portfolio_growth_df = pd.DataFrame()
    for main_index, main_row in sensex_df.iterrows():
        monthly_portfolio_value  = generate_portfolio(main_index, withdrawals)
        portfolio_growth_df.insert(main_index, main_row["Month"], monthly_portfolio_value)
    performance_at_swr[swr] = portfolio_growth_df

In [ ]:
def inflated_value(initial_value, timeperiod, monthly_inflation):
    return initial_value * (pow((1 + monthly_inflation ), timeperiod)) 

In [ ]:
## Calculate the inflated value of portfolio
inflated_corpus = list()
total_months = len(sensex_df.index) -1
for month  in sensex_df["Month"]:
    corpus_value = inflated_value(100, total_months, MONTHLY_INFLATION)
    inflated_corpus.append((month, corpus_value))
    total_months -= 1
inflated_corpus_df = pd.DataFrame.from_records(inflated_corpus, columns=["Month", "corpus"])
inflated_corpus_df

In [ ]:
## Current Value of Portfolio
def get_current_value(performace_df):
    current_value_df  = performace_df.tail(1)
    current_value_df = current_value_df.transpose()
    current_value_df.columns = ["corpus"]
    current_value_df['Month'] = current_value_df.index
    current_value_df = current_value_df[['Month', 'corpus']]
    current_value_df = current_value_df.reset_index(drop=True)
    return current_value_df
get_current_value(performance_at_swr[4])

In [ ]:
performance_list = list()
for swr in SWR_RANGES:
    current_value_df = get_current_value(performance_at_swr[swr])
    temp_current_value_df = current_value_df[:-EXCLUDE_RESULT]
    temp_inflated_corpus_df = inflated_corpus_df[:-EXCLUDE_RESULT]
    preservation_result =  temp_current_value_df['corpus'] - temp_inflated_corpus_df['corpus']
    preservation_result_df = pd.DataFrame(preservation_result)
    preservation_status = preservation_result_df.apply(lambda x: True if x['corpus'] >0 else False , axis=1)
    success = len(preservation_status[preservation_status == True].index)
    performance_list.append((swr, success/len(preservation_status)))

In [ ]:
performance_df = pd.DataFrame.from_records(performance_list, columns=["SWR", "Performance"])

In [ ]:
performance_df

In [ ]:
performance_df

In [ ]:
fig = px.line(performance_at_swr[4], title='SWR 4% - 1991 - 2020')
fig.show()

Years Failed

In [ ]:
current_value_df = get_current_value(performance_at_swr[4])
temp_current_value_df = current_value_df[:-EXCLUDE_RESULT]
temp_inflated_corpus_df = inflated_corpus_df[:-EXCLUDE_RESULT]
preservation_result =  temp_current_value_df['corpus'] - temp_inflated_corpus_df['corpus']
preservation_result


In [ ]:
preservation_result_df = pd.DataFrame(preservation_result)
preservation_result_df